# Meter los datos en la base de datos

Puedes ejecutar este notebook desde la linea de comandos con:

```bash
pushd chat
jupyter nbconvert --execute add_info_python.ipynb
popd chat
```

In [1]:
import os
from tqdm import tqdm
import sys
import chromadb
import chromadb.utils.embedding_functions as embedding_functions

print(f"Inicializando chromadb")
for i in tqdm(range(10)):
    pass 

print("done")
sys.exit(1)
print("continues")
chroma_client = chromadb.PersistentClient(path='db') # current folder
llama3_embedding_function = embedding_functions.OllamaEmbeddingFunction(
    url="http://localhost:11434/api/embeddings", # ollama running in the background
    model_name="llama3",
)

collection_glossary = chroma_client.create_collection(name="glossary",
                                                      embedding_function=llama3_embedding_function)
collection_turras = chroma_client.create_collection(name="turras",
                                                    embedding_function=llama3_embedding_function)


Inicializando chromadb


In [23]:
import csv
print("Glosario loader")
def glosario():
    file_path = os.path.join(os.getcwd(), '..', 'db', 'glosario.csv')
    
    with open(file_path, mode='r', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile, fieldnames=['word', 'definition', 'sources'], delimiter=',', quotechar='"')
        next(reader) # header
        for row in reader:
            yield {
                'word': row['word'],
                'definition': row['definition'],
                'sources': row['sources'] or ''
            }

glos_documents = []
glos_ids = []

for item in tqdm(glosario()):
    glos_documents.append(f"{item['word']}: {item['definition']}")
    glos_ids.append(item['word'])

63it [00:00, 38819.03it/s]


In [24]:
collection_glossary.add(documents=glos_documents, ids=glos_ids)

## Importar las turras

In [25]:
import json
import itertools
print(f"Turra loader")

def turras():
    file_path = os.path.join(os.getcwd(), '..', 'db', 'tweets.json')
    with open(file_path, mode='r', encoding='utf-8') as jsonfile:
        turras = json.load(jsonfile)
        total = len(turras)
        for turra in tqdm(j):
            url = "https://turrero.vercel.app/turra/" + turra['id']
            completa = '\n'.join(tuit for tuit in turra)
            yield (url, completa)
                
urls_generator, completas_generator = itertools.tee(turras())

In [ ]:
collection_turras.add({
    'documents': (completa or _, completa in completas_generator),
    'ids': (url for for url, _ in urls_generator),
})